In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import serial

import pygame
from pygame import *

WIN_WIDTH = 1600
WIN_HEIGHT = 960
HALF_WIDTH = int(WIN_WIDTH / 2)
HALF_HEIGHT = int(WIN_HEIGHT / 2)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
LGREEN = (60, 225, 60)
GREEN = (0, 255, 0)
DARKGR = (14, 122, 16)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREY = (100, 100, 100)
LGREY =(150, 150, 150)
DGREY =(50, 50,50)
SLATE = (30, 107, 122)
GOLD = (249, 238, 74)
PURPLE = (153, 4, 216)
PINK = (247, 128, 128)
DISPLAY = (WIN_WIDTH, WIN_HEIGHT)
DEPTH = 32
FLAGS = HWSURFACE|DOUBLEBUF

class MainMenu(object):
    def __init__(self):
        
        self.menu = pygame.Surface([WIN_WIDTH, WIN_HEIGHT])
        self.menu.fill(BLACK)
        self.rect = self.menu.get_rect()
        self.rect.x = 0
        self.rect.y = 0
        self.myfont = pygame.font.SysFont("monospace", 15)
        self.counter = 0
        self.ind = 0       
        self.start = False
        self.title = pygame.image.load('SerialPlot.png').convert()          
        self.buttonUnpressed = pygame.Surface([200, 25])
        self.buttonUnpressed.fill(LGREEN)

        self.buttonPressed = pygame.Surface([200, 25])
        self.buttonPressed.fill(DARKGR)

        self.button = self.buttonUnpressed
    def update(self, mouse, mouseclick):
        self.counter += 1
        if ((HALF_WIDTH-100)<mouse[0]<(HALF_WIDTH+100)) and ((HALF_HEIGHT+50)<mouse[1]<(HALF_HEIGHT+75)):
            self.button = self.buttonPressed
            if mouseclick:
                self.start = True
                
        else:
            self.button = self.buttonUnpressed
            
    def draw(self, screen):
        
        label2 = self.myfont.render(("Connect Arduino"), 1, (0,0,0))
        screen.blit(self.menu,(self.rect.x, self.rect.y))
        screen.blit(self.title, ((HALF_WIDTH-128), (HALF_HEIGHT-90) ))
        screen.blit(self.button, ((HALF_WIDTH-100), (HALF_HEIGHT+50))) 
        screen.blit(label2, ((HALF_WIDTH-75), (HALF_HEIGHT+54)))
        
        
class Arduino(object):
    def __init__(self):
        self.rawIn = None
    def connectArduino(self):

        self.found = False
        counter = 0
        while not self.found:
            try:
                
                string1 = ("/dev/ttyACM%s" % counter)
                self.rawIn = serial.Serial(string1, 9600)
                self.found = True
                
                
            except:
                counter += 1
                if counter == 20:
                    print('Could not Connect Arduino')
                    break
                    
#class Tickbox(object):
#    def __init__(self):
        
class Plotter(object):
    #Plotter class creates objects that handle live plotting of data
    def __init__(self, settings):
        self.settings = settings
        #initialize lists for point and axis storage, updates, and drawing
        self.points = []
        self.movingpoints = []
        self.vertaxes = []
        #set font for axis labels
        self.axisfont = pygame.font.SysFont("monospace", 8)
        #setup all surfaces in the graphing frame
        self.bg = pygame.Surface((WIN_WIDTH, WIN_HEIGHT)).convert()
        self.bg.fill(BLACK)
        
        self.vertaxsurf = pygame.Surface((2, WIN_HEIGHT-40)).convert()
        self.vertaxsurf.fill(GREY)
        
        self.graphrect = pygame.Surface((WIN_WIDTH-200, WIN_HEIGHT-40)).convert()
        self.graphrect.fill(LGREY)
    
        self.pointsurface = pygame.Surface((4,4)).convert()
        self.pointsurface.fill(RED)
    
        self.yAxis = pygame.Surface((1, WIN_HEIGHT-40)).convert()
        self.yAxis.fill(DGREY)
    
        self.xAxis = pygame.Surface((WIN_WIDTH - 200, 1)).convert()
        self.xAxis.fill(DGREY)
        #render axis labels
        self.axisLabels = []
    
        for x in range(-512, 528, 16):
            self.axisLabels.append(self.axisfont.render((str(x)), 1, BLACK))
        
    def update(self, iterator):
        #create new point based on points[-1] and iterator. add to list of moving points
        self.movingpoints.append(MovingPoint(((iterator*self.settings.speed)+20), int(self.points[-1]), 10, 10))
        #add axis if iterator is multiple of axfreq
        if iterator%(self.settings.axfreq) == 0:
            self.vertaxes.append(pygame.Rect(((iterator*self.settings.speed)+20), 20, 1, WIN_HEIGHT-40))
        #movement code that begins to execute once points have reached edge
        if self.movingpoints[-1].x > 1350:
            self.movingpoints[-1].x = 1350
            
            for point in self.movingpoints:
                point.move_ip(-self.settings.speed, 0)
                
                
                if point.x < 20:
                
                    self.movingpoints.remove(point)
            for ax in self.vertaxes:
                
                ax.move_ip(-self.settings.speed, 0)
                if ax.x < 20:
                    try:
                        self.vertaxes.remove(ax)
                    except:
                        pass
        try:
            if (self.vertaxes[-1].x > 1350):
                self.vertaxes[-1].x = 1350
        except:
            pass

    def draw(self, screen):
        #Plotting 
        screen.blit(self.bg, (0,0))
        screen.blit(self.graphrect, [20, 20])
        screen.blit(self.xAxis, [20, HALF_HEIGHT])
        screen.blit(self.yAxis, [40, 20])
        
        for point in self.movingpoints:
            screen.blit(self.pointsurface, [point.x+20, (point.y*(12/16))+HALF_HEIGHT])
        for ax in self.vertaxes:
            screen.blit(self.vertaxsurf, [ax.x+20, 20])
            
        
            #screen.blit(axisspacinglabel, [WIN_WIDTH -170, HALF_HEIGHT+150])
        spacer1 = -384

        for x in self.axisLabels:
            
            screen.blit(x, [20, HALF_HEIGHT+1+spacer1])
            spacer1 += 12
                 
class MovingPoint(Rect):
    def __init__(self, left, top, width, height):
        super().__init__(left, top, width, height)
        self.vel = 1 

        
class PlotSettingsFrame(object):
    #Plotsettingsframe class creates a frame object with some controls and information
    def __init__(self, iterator, timeNow, lastpoint):
        self.myfont = pygame.font.SysFont("monospace", 15)
        self.myfont2 = pygame.font.SysFont("monospace", 12)
        self.buttonrect = pygame.Surface((WIN_WIDTH-870, WIN_HEIGHT-40))
        self.buttonrect.convert()
        self.buttonrect.fill(SLATE)
        
        self.buttonimages = []
        
        self.buttonimages.append(pygame.image.load('button-2.png').convert_alpha())
        self.buttonimages.append(pygame.image.load('button-1.png').convert_alpha())
        self.buttonimages.append(pygame.image.load('button-3.png').convert_alpha())
        self.buttonimage = self.buttonimages[1]
        self.buttonimage2 = self.buttonimages[1]
        self.iterator = iterator
        self.speed = 1
        self.axfreq = 50
        self.timeHolder= 0
        self.timeHolder = self.timeHolder + timeNow
        timestring = str(self.timeHolder)
        timestring = timestring[:4]
        self.timelabel = self.myfont.render(("Time = %s" % timestring), 1, WHITE)
        self.speedlabel = self.myfont.render(("Zoom Level: %s" % self.speed),1, WHITE)
        self.axisspacinglabel = self.myfont.render(("Axis Period: %s s"%(self.axfreq*timeNow)), 1, WHITE)
        self.buttonlabel1 = self.myfont2.render(("Change Zoom"), 1, BLACK)
        self.buttonlabel2 = self.myfont2.render(("Change Ax Freq"), 1, BLACK)
        self.settingslabel = self.myfont.render(("Settings"), 1, WHITE)

    def update(self, iterator, timeNow, lastpoint, mouse, mouseclick, clickstore):
        if ((WIN_WIDTH-170)<mouse[0]<(WIN_WIDTH-42)) and ((HALF_HEIGHT-48) <mouse[1]<(HALF_HEIGHT)):
            self.buttonimage = self.buttonimages[0]
            if mouseclick:
                self.buttonimage = self.buttonimages[2]
        else:
            self.buttonimage = self.buttonimages[1]
        if ((WIN_WIDTH-170)<mouse[0]<(WIN_WIDTH-42)) and ((HALF_HEIGHT-106) <mouse[1]<(HALF_HEIGHT-56)):
            self.buttonimage2 = self.buttonimages[0]
            if mouseclick:
                self.buttonimage2 = self.buttonimages[2]
        else:
            self.buttonimage2 = self.buttonimages[1]
        

        self.iterator = iterator
        self.timeHolder = self.timeHolder + timeNow
        timestring = str(self.timeHolder)
        timestring = timestring[:4]
        self.lastpointlabel = self.myfont.render(("Last Point: %s" % lastpoint), 1, WHITE)
        self.timelabel = self.myfont.render(("Time = %s" % timestring), 1, WHITE)
        self.speedlabel = self.myfont.render(("Zoom Level: %s" % self.speed),1, WHITE)
        self.axisspacinglabel = self.myfont.render(("Axis Period: %s s"%(self.axfreq*timeNow)), 1, WHITE)
    def speedchange(self, mouse, mouseclick):
        
        if ((WIN_WIDTH-170)<mouse[0]<(WIN_WIDTH-42)) and ((HALF_HEIGHT-48) <mouse[1]<(HALF_HEIGHT)):
            if mouseclick == 1:
                self.speed+=1
            elif mouseclick == 3:
                if self.speed > 1:
                    self.speed-=1
            else:
                pass
        
    def axfreqchange(self,mouse,mouseclick):
        if ((WIN_WIDTH-170)<mouse[0]<(WIN_WIDTH-42)) and ((HALF_HEIGHT-106) <mouse[1]<(HALF_HEIGHT-58)):
            if mouseclick == 1:
                self.axfreq+=1
            elif mouseclick == 3:
                if self.axfreq > 1:
                    self.axfreq-=1
            else:
                pass
    def draw(self, screen):
        
        
        screen.blit(self.buttonrect, [WIN_WIDTH - 180, 20])
        screen.blit(self.settingslabel, [WIN_WIDTH - 170, 30])
        screen.blit(self.buttonimage,[WIN_WIDTH - 170, HALF_HEIGHT-48] )
        screen.blit(self.buttonimage2, [WIN_WIDTH - 170, HALF_HEIGHT-106])
        screen.blit(self.buttonlabel1, [WIN_WIDTH - 156, HALF_HEIGHT-28])
        screen.blit(self.buttonlabel2, [WIN_WIDTH - 160, HALF_HEIGHT-86])
        screen.blit(self.lastpointlabel, [WIN_WIDTH - 170, HALF_HEIGHT+25])
        screen.blit(self.timelabel, [WIN_WIDTH - 170, HALF_HEIGHT + 50])
        screen.blit(self.speedlabel,[WIN_WIDTH -170, HALF_HEIGHT+75] )
        screen.blit(self.axisspacinglabel, [WIN_WIDTH -170, HALF_HEIGHT+100])
        
        
class Button(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__(self)
        
        self.image = pygame.image.load('button-2.png').convert_alpha()
        self.hoverimage = pygame.image.load('button-1.png').convert_alpha()
        self.pressimage = pygame.image.load('button-3.png').convert_alpha()
        self.rect = self.image.get_rect()
        self.output = self.image
        

        
def main():
    
    pygame.init()
    clock = pygame.time.Clock()
    screen = pygame.display.set_mode(DISPLAY, FLAGS, DEPTH)
    pygame.display.set_caption("Serial Plotter")
    on = True
    mouseclick = False
    mouse = 0
    timeNow = clock.get_time()/1000
    iterator = 0
    menuflag = True
    lastpoint = 0
    clickstore = 0
    clickflag = False
    first = True
    settings = PlotSettingsFrame(iterator, timeNow, lastpoint)
    plot = Plotter(settings)
    menu = MainMenu()
    first = True
    while(on):
        
        mouse = pygame.mouse.get_pos()
        for e in pygame.event.get():
            if e.type == QUIT: 
                on = False
            if e.type == KEYDOWN and e.key == K_ESCAPE:
                on= False
            if e.type == pygame.MOUSEBUTTONDOWN:
                clickstore = e.button
                mouseclick = True
                settings.speedchange(mouse, clickstore)
                settings.axfreqchange(mouse,clickstore)
            if e.type == pygame.MOUSEBUTTONUP:
                mouseclick = False
                clickflag = False
        if menuflag:
            menu.update(mouse,mouseclick)
            menu.draw(screen)
            if menu.start:
                menuflag= False
        else:
            
            if (first):
                
                try:
                    
                    ard = Arduino()
                    ard.connectArduino()
                    
                    if not ard.found:
                        
                        break
                    first = False
                    print('Connected')
                except:
                    print('Could not connect')
                    menuflag = True
            else:
                pass
            
            datain = ard.rawIn.readline()
            datain = datain.decode('UTF-8')
            datain = datain[:-1]
            plot.points.append(datain)
        
            lastpoint = plot.points[-1]

            timeNow = clock.get_time()/1000

            settings.update(iterator, timeNow, lastpoint, mouse, mouseclick, clickstore)
            plot.update(iterator)
            plot.draw(screen)
            settings.draw(screen)
            
            iterator += 1
        pygame.display.flip()
        clock.tick(100)
    pygame.quit()
    
    
if __name__ == "__main__":
    main()
